# Reshaping data for model
### In this file, we will take the data that has been gathered and harmonized and we will fit it, create scenarios, and save it as a structured array for the model. Since we would like to keep the flexibility with excel, we will also save it in an ODYM compatible format and create a file that can do the reverse: if teh excel file is eddited, so is the array. This will be a separate script

In [1]:
# Load a local copy of the current ODYM branch:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import xlrd
import pylab
from copy import deepcopy
import logging as log
import xlwt
import tqdm
import math
from scipy.stats import norm
from openpyxl import *
from scipy.optimize import curve_fit
from scipy.stats import gompertz
from sklearn.metrics import r2_score
from logistic import logistic as logistic
### Preamble
os.getcwd()
os.chdir("..")
os.chdir("..")
# add ODYM module directory to system path, relative
MainPath = os.path.join(os.getcwd(), 'odym', 'modules')
sys.path.insert(0, MainPath)

# add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.getcwd(), 'odym', 'modules'))

# Specify path to dynamic stock model and to datafile, relative
DataPath = os.path.join( 'docs', 'files')

# Specify path to dynamic stock model and to datafile, absolute
DataPath = os.path.join(os.getcwd(), 'docs', 'Files')

import ODYM_Classes as msc # import the ODYM class file
import ODYM_Functions as msf # import the ODYM function file
import dynamic_stock_model as dsm # import the dynamic stock model library

# Initialize loggin routine
log_verbosity = eval("log.DEBUG")
log_filename = 'LogFileTest.md'
[Mylog, console_log, file_log] = msf.function_logger(log_filename, os.getcwd(),
                                                     log_verbosity, log_verbosity)
Mylog.info('### 1. - Initialize.')

#Read main script parameters
#Load project-specific config file
ProjectSpecs_ConFile = 'ODYM_Config_Vehicle_System.xlsx'
Model_Configfile     = xlrd.open_workbook(os.path.join(DataPath, ProjectSpecs_ConFile))
ScriptConfig         = {'Model Setting': Model_Configfile.sheet_by_name('Config').cell_value(3,3)} # Dictionary with config parameters
Model_Configsheet    = Model_Configfile.sheet_by_name('Setting_' + ScriptConfig['Model Setting'])

Name_Scenario        = Model_Configsheet.cell_value(3,3)
print(Name_Scenario)

#Read control and selection parameters into dictionary
ScriptConfig         = msf.ParseModelControl(Model_Configsheet,ScriptConfig)

Mylog.info('Read and parse config table, including the model index table, from model config sheet.')
IT_Aspects,IT_Description,IT_Dimension,IT_Classification,IT_Selector,IT_IndexLetter,\
PL_Names,PL_Description,PL_Version,PL_IndexStructure,PL_IndexMatch,PL_IndexLayer,\
PrL_Number,PrL_Name,PrL_Comment,PrL_Type,ScriptConfig = msf.ParseConfigFile(Model_Configsheet,ScriptConfig,Mylog)    

class_filename       = 'ODYM_Classifications_Master_Vehicle_System.xlsx'
Classfile            = xlrd.open_workbook(os.path.join(DataPath,class_filename))
Classsheet           = Classfile.sheet_by_name('MAIN_Table')
MasterClassification = msf.ParseClassificationFile_Main(Classsheet,Mylog)

Mylog.info('Define model classifications and select items for model classifications according to information provided by config file.')
ModelClassification  = {} # Dict of model classifications
for m in range(0,len(IT_Aspects)):
    ModelClassification[IT_Aspects[m]] = deepcopy(MasterClassification[IT_Classification[m]])
    EvalString = msf.EvalItemSelectString(IT_Selector[m],len(ModelClassification[IT_Aspects[m]].Items))
    if EvalString.find(':') > -1: # range of items is taken
        RangeStart = int(EvalString[0:EvalString.find(':')])
        RangeStop  = int(EvalString[EvalString.find(':')+1::])
        ModelClassification[IT_Aspects[m]].Items = ModelClassification[IT_Aspects[m]].Items[RangeStart:RangeStop]           
    elif EvalString.find('[') > -1: # selected items are taken
        ModelClassification[IT_Aspects[m]].Items = [ModelClassification[IT_Aspects[m]].Items[i] for i in eval(EvalString)]
    elif EvalString == 'all':
        None
    else:
        Mylog.error('Item select error for aspect ' + IT_Aspects[m] + ' were found in datafile.')
        break

# Define model index table and parameter dictionary
Mylog.info('### 2.2 - Define model index table and parameter dictionary')
Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End   = int(max(ModelClassification['Time'].Items))
Model_Duration   = Model_Time_End - Model_Time_Start + 1

Mylog.info('Define index table dataframe.')
IndexTable = pd.DataFrame({'Aspect'        : IT_Aspects,  # 'Time' and 'Element' must be present!
                           'Description'   : IT_Description,
                           'Dimension'     : IT_Dimension,
                           'Classification': [ModelClassification[Aspect] for Aspect in IT_Aspects],
                           'IndexLetter'   : IT_IndexLetter})  # Unique one letter (upper or lower case) indices to be used later for calculations.

# Default indexing of IndexTable, other indices are produced on the fly
IndexTable.set_index('Aspect', inplace=True)

# Add indexSize to IndexTable:
IndexTable['IndexSize'] = pd.Series([len(IndexTable.Classification[i].Items) for i in range(0, len(IndexTable.IndexLetter))],
                                    index=IndexTable.index)

# list of the classifications used for each indexletter
IndexTable_ClassificationNames = [IndexTable.Classification[i].Name for i in range(0, len(IndexTable.IndexLetter))]

# Define dimension sizes
Nt = len(IndexTable.Classification[IndexTable.index.get_loc('Time')].Items)
Nc = len(IndexTable.Classification[IndexTable.index.get_loc('Age-cohort')].Items)
Ng = len(IndexTable.Classification[IndexTable.index.get_loc('Good')].Items)
Nr = len(IndexTable.Classification[IndexTable.index.get_loc('Region')].Items)
Ne = len(IndexTable.Classification[IndexTable.index.get_loc('Element')].Items)
Nb = len(IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items)
# Nk = len(IndexTable.Classification[IndexTable.index.get_loc('Capacity')].Items)
Np = len(IndexTable.Classification[IndexTable.index.get_loc('Battery_Parts')].Items)
Ns = len(IndexTable.Classification[IndexTable.index.get_loc('Size')].Items)
Nh = len(IndexTable.Classification[IndexTable.index.get_loc('Recycling_Process')].Items)
# Nv = len(IndexTable.Classification[IndexTable.index.get_loc('Make')].Items)
NS = len(IndexTable.Classification[IndexTable.index.get_loc('Scenario')].Items)

INFO (<ipython-input-1-d25a58768329> <<module>>): ### 1. - Initialize.
INFO (<ipython-input-1-d25a58768329> <<module>>): Read and parse config table, including the model index table, from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read parameter list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read process list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model run control from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model output control from model config sheet.


Vehicle stock model for Global fleet


INFO (ODYM_Functions.py <ParseClassificationFile_Main>): End of file or formatting error while reading the classification file in column 20. Check if all classifications are present. If yes, you are good to go!
INFO (<ipython-input-1-d25a58768329> <<module>>): Define model classifications and select items for model classifications according to information provided by config file.
INFO (<ipython-input-1-d25a58768329> <<module>>): ### 2.2 - Define model index table and parameter dictionary
INFO (<ipython-input-1-d25a58768329> <<module>>): Define index table dataframe.
DEBUG (pyplot.py <switch_backend>): Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Preparing battery parts

This file needs major revision and the data assumptions need to be well documented. 

In [2]:
df = pd.read_excel('/Users/fernaag/Library/CloudStorage/Box-Box/BATMAN/Data/Database/data/01_raw_data/Xi_et_al_model_2020/Material_content_new.xlsx', sheet_name='share per part').drop('Comment', axis=1)

In [3]:
df.head()

,drive_train,segment,part,LFP,NCA,NCM111,NCM523,NCM622,NCM622-Graphite (Si),NCM811-Graphite (Si),NCM955-Graphite (Si),Li-Sulphur,Li-Air
0,BEV,Small,Modules,0.631370,0.737702,0.743918,0.743967,0.741366,0.731219,0.728257,0.727652,0.361643,0.319981
1,BEV,Small,Case (pack),0.268630,0.162298,0.156082,0.156033,0.158634,0.168781,0.171743,0.172348,0.538357,0.580019
2,BEV,Small,"BMS, wiring, and equipment",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
3,BEV,Medium,Modules,0.658333,0.783521,0.782660,0.784006,0.785044,0.775409,0.776963,0.777385,0.361390,0.319362
4,BEV,Medium,Case (pack),0.241667,0.116479,0.117340,0.115994,0.114956,0.124591,0.123037,0.122615,0.538610,0.580638


In [4]:
df = df.melt(['drive_train', 'segment', 'part'])

In [5]:
df.head()

,drive_train,segment,part,variable,value
0,BEV,Small,Modules,LFP,0.631370
1,BEV,Small,Case (pack),LFP,0.268630
2,BEV,Small,"BMS, wiring, and equipment",LFP,0.100000
3,BEV,Medium,Modules,LFP,0.658333
4,BEV,Medium,Case (pack),LFP,0.241667


In [6]:
PartsArray = np.zeros((Ng, Ns, Nb, Np))

In [7]:
for m in range(0, len(df.segment.values)):
        SizePosition = IndexTable.Classification[IndexTable.index.get_loc('Size')].Items.index(df.segment.iloc[m])
        DTPosition = IndexTable.Classification[IndexTable.index.get_loc('Good')].Items.index(df.drive_train.iloc[m])
        ChemistryPosition= IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items.index(df.variable.iloc[m])
        PartPosition= IndexTable.Classification[IndexTable.index.get_loc('Battery_Parts')].Items.index(df.part.iloc[m])
        PartsArray[DTPosition, SizePosition, ChemistryPosition, PartPosition] = df.value.iloc[m]

In [8]:
# Adding generic shares for other chemistries
generic_weights = [0.7,0.2,0.1]
for g in range(Ng):
    for b in range(Nb):
        for s in range(Ns):
            for p in range(Np):
                if PartsArray[g,s,b,p] == 0:
                    PartsArray[g,s,b,p] = generic_weights[p]

In [9]:
np.save('/Users/fernaag/Library/CloudStorage/Box-Box/BATMAN/Data/Database/data/03_scenario_data/global_model/batteryParts/batteryParts_motorEnergy.npy', PartsArray, allow_pickle=True)

In [10]:
PartsArray

array([[[[0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ]],

        [[0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0.7       , 0.2       , 0.1       ],
         [0